<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>Neural ProPhet - V6.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt
#
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
productos_descripcion = pd.read_csv('data/tb_productos_descripcion.txt', delimiter='\t')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1600)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())
print(productos_descripcion.head())
productos_descripcion.info()


   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [3]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [4]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
data.info()
data

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [5]:
# Definir una función de agregación personalizada
def agg_func(x):
    return pd.Series({
        'plan_precios_cuidados': x['plan_precios_cuidados'].iloc[0],  # Tomar el primer valor
        'cust_request_qty': x['cust_request_qty'].sum(),
        'cust_request_tn': x['cust_request_tn'].sum(),
        'tn': x['tn'].sum()
    })

# Agrupar y agregar
data_agrupada = data.groupby(['periodo', 'customer_id', 'product_id']).apply(agg_func).reset_index()

# Mostrar información sobre el nuevo DataFrame
print(data_agrupada.info())

# Mostrar las primeras filas del nuevo DataFrame
print(data_agrupada.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293481 entries, 0 to 2293480
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  float64
 4   cust_request_qty       float64
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(4), int64(3)
memory usage: 122.5 MB
None
   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn         tn
0   201701        10001       20001                    0.0              11.0         99.43861   99.43861
1   201701        10001       20002                    0.0              17.0         90.13504   87.64856
2   201701        10001       20003                    0.0              25.0        100.21284  100.21284
3   201701        10001       20004                    0.0              13.0         21

In [6]:
def filter_products(data, product_id_min, product_id_max):
    """
    Filtra el DataFrame para incluir solo los productos dentro del rango especificado.
    
    Args:
    data (pd.DataFrame): El DataFrame a filtrar.
    product_id_min (int): El ID de producto mínimo a incluir.
    product_id_max (int): El ID de producto máximo a incluir.
    
    Returns:
    pd.DataFrame: DataFrame filtrado.
    """
    filtered = data[(data['product_id'] >= product_id_min) & (data['product_id'] <= product_id_max)]
    
    print(f"Productos seleccionados: {filtered['product_id'].nunique()}")
    print(f"Rango de productos: {product_id_min} - {product_id_max}")
    print(f"Registros en el DataFrame filtrado: {len(filtered)}")
    
    return filtered

# Ejemplo de uso
product_id_min = 20000
product_id_max = 20002
filtered_data = filter_products(data_agrupada, product_id_min, product_id_max)

# Mostrar los primeros registros del DataFrame filtrado
print("\nPrimeros registros del DataFrame filtrado:")
print(filtered_data.head())

# Mostrar un resumen de los productos incluidos
print("\nResumen de productos incluidos:")
product_summary = filtered_data.groupby('product_id').agg({
    'tn': 'sum',
    'cust_request_qty': 'sum',
    'periodo': 'nunique'
}).reset_index()
product_summary.columns = ['product_id', 'total_tn', 'total_qty', 'num_periodos']
print(product_summary)

Productos seleccionados: 2
Rango de productos: 20000 - 20002
Registros en el DataFrame filtrado: 12172

Primeros registros del DataFrame filtrado:
     periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn         tn
0     201701        10001       20001                    0.0              11.0         99.43861   99.43861
1     201701        10001       20002                    0.0              17.0         90.13504   87.64856
425   201701        10002       20001                    0.0              17.0         38.68301   35.72806
426   201701        10002       20002                    0.0              25.0         23.62160   23.62160
870   201701        10003       20001                    0.0              17.0        143.49426  143.49426

Resumen de productos incluidos:
   product_id     total_tn  total_qty  num_periodos
0       20001  50340.39558    15708.0            36
1       20002  36337.25439    15222.0            36


In [7]:
print(product_summary.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    2 non-null      int64  
 1   total_tn      2 non-null      float64
 2   total_qty     2 non-null      float64
 3   num_periodos  2 non-null      int64  
dtypes: float64(2), int64(2)
memory usage: 196.0 bytes
None


In [8]:
print(filtered_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12172 entries, 0 to 2293461
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   periodo                12172 non-null  int64  
 1   customer_id            12172 non-null  int64  
 2   product_id             12172 non-null  int64  
 3   plan_precios_cuidados  12172 non-null  float64
 4   cust_request_qty       12172 non-null  float64
 5   cust_request_tn        12172 non-null  float64
 6   tn                     12172 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 760.8 KB
None


In [9]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [10]:
# Convertir 'periodo' a datetime
filtered_data['ds'] = pd.to_datetime(filtered_data['periodo'].astype(str), format='%Y%m')

# Preparar los regresores
regresores = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn']

In [11]:
# Función para crear lags
def create_lags(df, columns, lags):
    for col in columns:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df.groupby(['product_id', 'customer_id'])[col].shift(lag)
    return df

# Crear lags
filtered_data = create_lags(filtered_data, regresores + ['tn'], [3, 12])

# Normalizar los datos
#scaler = StandardScaler()
#columns_to_scale = regresores + [col for col in filtered_data.columns if 'lag' in col]
#filtered_data[columns_to_scale] = scaler.fit_transform(filtered_data[columns_to_scale])

In [12]:
def train_and_predict(group):
    try:
        # Filtrar regresores con valores únicos
        regresores_validos = [col for col in regresores + [c for c in group.columns if 'lag' in c] 
                              if group[col].nunique() > 1]
        
        model = NeuralProphet(
            n_forecasts=2,
            yearly_seasonality=True,
            seasonality_mode='multiplicative',
            n_lags=12,
            batch_size=16,
            epochs=1000,
            learning_rate=0.1,
            loss_func='Huber'
        )
        
        # Agregar estacionalidad trimestral manualmente
        model.add_seasonality(name='quarterly', period=91.25, fourier_order=5)
        
        # Renombrar 'tn' a 'y' para NeuralProphet
        df = group[['ds', 'tn'] + regresores_validos].dropna()
        df = df.rename(columns={'tn': 'y'})
        
        if len(df) < 24:  # Asegurar que haya suficientes datos para entrenar
            return np.nan
        
        # Imprimir estadísticas de los datos de entrenamiento
        print(f"Estadísticas para product_id {group['product_id'].iloc[0]}, customer_id {group['customer_id'].iloc[0]}:")
        print(df['y'].describe())
        
        # Crear datos futuros para los regresores
        future_periods = 2
        future_df = pd.DataFrame({
            'ds': pd.date_range(start=df['ds'].max(), periods=future_periods+1, freq='M')[1:],
            'y': [np.nan] * future_periods
        })
        
        for reg in regresores_validos:
            if reg in df.columns:
                model.add_future_regressor(reg)
                # Usar el último valor conocido para los datos futuros
                future_df[reg] = df[reg].iloc[-1]
        
        model.fit(df, freq='M')
        
        future = model.make_future_dataframe(df, periods=future_periods, n_historic_predictions=len(df))
        future = pd.concat([future, future_df[regresores_validos]], axis=1)
        
        forecast = model.predict(future)
        
        prediction = forecast['yhat2'].iloc[-1]
        
        # Si la predicción es cero, usar el último valor no cero conocido
        if prediction == 0:
            last_non_zero = df['y'].replace(0, np.nan).dropna().iloc[-1] if len(df['y'].replace(0, np.nan).dropna()) > 0 else df['y'].mean()
            prediction = last_non_zero
        
        print(f"Predicción para product_id {group['product_id'].iloc[0]}, customer_id {group['customer_id'].iloc[0]}: {prediction}")
        
        return prediction
    except Exception as e:
        print(f"Error en product_id {group['product_id'].iloc[0]}, customer_id {group['customer_id'].iloc[0]}: {str(e)}")
        return np.nan

In [13]:
# Filtrar grupos con suficientes datos antes de aplicar train_and_predict
def has_enough_data(group):
    return len(group.dropna()) >= 24

tqdm.pandas()
predictions = filtered_data.groupby(['product_id', 'customer_id']).filter(has_enough_data).groupby(['product_id', 'customer_id']).progress_apply(train_and_predict)

# Manejar los NaN en las predicciones
predictions = predictions.dropna()

# Sumar las predicciones por product_id
final_predictions = predictions.groupby('product_id').sum().reset_index()
final_predictions.columns = ['product_id', 'tn']

# Imprimir las predicciones finales
print("Predicciones finales:")
print(final_predictions)

# Verificar si hay predicciones de cero o muy bajas
low_predictions = final_predictions[final_predictions['tn'] < 100]  # Ajusta este umbral según tus necesidades
if not low_predictions.empty:
    print("Advertencia: Se encontraron predicciones bajas para los siguientes product_id:")
    print(low_predictions)

# Guardar las predicciones en un archivo CSV
final_predictions.to_csv('NeuralProphet.v6.6.csv', index=False)
print("Predicciones guardadas en 'NeuralProphet.v6.6.csv'")

# Imprimir información adicional de depuración
print("Forma final de final_predictions:", final_predictions.shape)
print("Columnas finales en final_predictions:", final_predictions.columns.tolist())

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.


Estadísticas para product_id 20001, customer_id 10001:
count     24.000000
mean     200.668123
std      111.671781
min       33.639910
25%      125.175063
50%      178.124590
75%      254.982197
max      439.906470
Name: y, dtype: float64


Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
 18%|███████████████                                                                    | 2/11 [00:15<01:08,  7.62s/it]WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override 

Error en product_id 20001, customer_id 10001: Future values of all user specified regressors not provided
Estadísticas para product_id 20001, customer_id 10004:
count     24.000000
mean     210.812270
std      120.659469
min       34.648100
25%      101.012135
50%      193.376250
75%      288.325250
max      469.457610
Name: y, dtype: float64


...

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [01:48<00:00, 10.42s/it]WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override 

Error en product_id 20002, customer_id 10008: Future values of all user specified regressors not provided
Estadísticas para product_id 20002, customer_id 10159:
count    24.000000
mean      1.407587
std       1.062401
min       0.044640
25%       0.842595
50%       1.227625
75%       1.632183
max       4.653820
Name: y, dtype: float64


Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [95.833]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [01:59<00:00, 10.90s/it]

Error en product_id 20002, customer_id 10159: Future values of all user specified regressors not provided
Predicciones finales:
Empty DataFrame
Columns: [product_id, tn]
Index: []
Predicciones guardadas en 'NeuralProphet.v6.6.csv'
Forma final de final_predictions: (0, 2)
Columnas finales en final_predictions: ['product_id', 'tn']


<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 